<a href="https://colab.research.google.com/github/seyed-mohammadreza-mousavi/Retinal-Vessel-Segmentation_A-Computer-Vision-Technique/blob/main/Welcome_To_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing face-recognition package

In [ ]:
!pip install face-recognition

Importing the package and detecting faces in the image

In [4]:
import face_recognition
image = face_recognition.load_image_file("o.jpg")
face_locations = face_recognition.face_locations(image)

Detecting face landmarks in the given image like chins and eyes and ... .

In [7]:
import face_recognition
image = face_recognition.load_image_file("b.jpg")
face_landmarks_list = face_recognition.face_landmarks(image)

Here we can recognize different images relate to the same person or not

In [14]:
import face_recognition
known_image = face_recognition.load_image_file("b.jpg")
unknown_image = face_recognition.load_image_file("b2.jpg")

biden_encoding = face_recognition.face_encodings(known_image)[0]
unknown_encoding = face_recognition.face_encodings(unknown_image)[0]

results = face_recognition.compare_faces([biden_encoding], unknown_encoding)